In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gmaps  # conda install -c conda-forge gmaps
import os
import scipy.io
import utm
%matplotlib inline
gmaps.configure(api_key=os.environ['GOOGLE_API_KEY']) # http://jupyter-gmaps.readthedocs.io/en/latest/authentication.html

In [ ]:
# Load the .mat file with the data
i_driver = 0  # [0-19]
data = scipy.io.loadmat('data.mat')
names = data['data'].dtype.names
d = {name: data['data'][0][name][0].T[0] for name in names}

In [ ]:
# Only take first lap
for key in d:
    if len(d[key].shape) == 1:
        if d[key].shape[0] > d['lap2'][0]:
            d[key] = d[key][0:d['lap2'][0]-1]

In [ ]:
# Compute UTM coordinates
zonenumber = 31
n = len(d['latitude'])
x = np.zeros(n)
y = np.zeros(n)
for i in range(n):
    x[i], y[i], _, _ = utm.conversion.from_latlon(d['latitude'][i], d['longitude'][i], zonenumber)
d['x'], d['y'] = x, y

## Show route on map

In [ ]:
fig = gmaps.figure()
latlon = np.array([x[0::10] for x in [d['latitude'], d['longitude']]]).T
fig.add_layer(gmaps.heatmap_layer(latlon, max_intensity=1, point_radius=3, gradient=[(0, 0, 0, 0), (0, 0, 100, 1)]))
fig

## Find highway type and speed limit

In [ ]:
# Define the ports
ports = [[51.474087, 5.625334, 'unclassified', 50],
        [51.475764, 5.631662, 'tertiary', 50],
        [51.474687, 5.633069, 'primary', 50],
        [51.473146, 5.626973, 'primary', 70],
        [51.471701, 5.609417, 'trunk', 100],
        [51.465868, 5.586438, 'motorway', 130],  # Nowadays, this is 100
        [51.456499, 5.539918, 'primary', 80],
        [51.454694, 5.523736, 'primary', 70],
        [51.447876, 5.502409, 'primary', 50],
        [51.448012, 5.499805, 'primary', 70],
        [51.452863, 5.486742, 'primary', 50],
        [51.453845, 5.485508, 'primary', 70],
        [51.481708, 5.489687, 'primary', 50],
        [51.485662, 5.490834, 'secondary', 50],
        [51.483252, 5.454073, 'tertiary', 30],
        [51.482943, 5.433376, 'primary', 70],
        [51.461943, 5.454222, 'primary', 50],
        [51.451746, 5.464362, 'secondary', 50],
        [51.438750, 5.468962, 'tertiary', 50],
        [51.438785, 5.467723, 'tertiary', 30],
        [51.435833, 5.451989, 'primary', 70],
        [51.424038, 5.492663, 'primary', 50],
        [51.415295, 5.497910, 'primary', 70],
        [51.413424, 5.498853, 'trunk', 70],
        [51.406835, 5.501793, 'trunk', 50],
        [51.404468, 5.504141, 'motorway_link', 120],
        [51.404908, 5.511230, 'motorway', 120],
        [51.406685, 5.561949, 'motorway_link', 80],
        [51.405968, 5.561648, 'secondary', 80],
        [51.407584, 5.560872, 'secondary', 50],
        [51.410741, 5.558681, 'tertiary', 50],
        [51.417679, 5.557647, 'secondary', 50],
        [51.431485, 5.580010, 'secondary', 80],
        [51.437847, 5.599852, 'secondary', 50],
        [51.448061, 5.619450, 'secondary', 80],
        [51.451768, 5.622338, 'secondary', 50],
        [51.460720, 5.631917, 'tertiary', 50],
        [51.475784, 5.631499, 'unclassified', 50]]
ports = [{'lat': x[0], 'lon': x[1], 'highway': x[2], 'speedlimit': x[3]} for x in ports]

In [ ]:
# Define the numbers for the highway labels
highway_labels = {'unclassified': 0,
                  'motorway': 1,
                  'trunk': 2,
                  'primary': 3,
                  'secondary': 4,
                  'tertiary': 5,
                  'motorway_link': 6}

In [ ]:
speedlim = 50  # Start with speed limit of 50
road_type = highway_labels['unclassified']

dist = np.zeros(n)
d['road_type'] = np.zeros(n)
d['road_type'][0] = road_type
d['speedlim'] = np.zeros(n)
d['speedlim'][0] = speedlim  
iport = 0
portx, porty, _, _ = utm.conversion.from_latlon(ports[iport]['lat'], ports[iport]['lon'], zonenumber)
dist[0] = np.hypot(portx - d['x'][0], porty - d['y'][0])

# Now loop through all datapoints and change if distance to port increases
for i in range(1, n):
    dist[i] = np.hypot(portx - d['x'][i], porty - d['y'][i])
    if dist[i] > dist[i-1] and dist[i] < 30:
        # We crossed a port
        road_type = highway_labels[ports[iport]['highway']]
        speedlim = ports[iport]['speedlimit']
        if iport + 1 < len(ports):
            iport += 1
        portx, porty, _, _ = utm.conversion.from_latlon(ports[iport]['lat'], ports[iport]['lon'], zonenumber)
    d['road_type'][i] = road_type
    d['speedlim'][i] = speedlim

In [ ]:
# Show speed limit on map
dist_points = 10  # [m]
i = [0]
for j in range(1, n):
    if np.hypot(d['x'][j] - d['x'][i[-1]], d['y'][j] - d['y'][i[-1]]) > dist_points:
        i.append(j)
idx_same_dist = i.copy()
fig = gmaps.figure()
latlon = np.array([d['latitude'][idx_same_dist], d['longitude'][idx_same_dist]]).T
fig.add_layer(gmaps.heatmap_layer(latlon, weights=d['speedlim'][idx_same_dist]/5000))
fig

In [ ]:
# Show the road type in different colors
fig = gmaps.figure()
latlon = np.array([d['latitude'][idx_same_dist], d['longitude'][idx_same_dist]]).T
fig.add_layer(gmaps.heatmap_layer(latlon, weights=1/(d['road_type'][idx_same_dist]+1)/20))
fig